Define values for Data Points

In [22]:
!pip install numpy

In [51]:
def value_func(i):
    return -2+0.2*i

def value_gen(max_val):
#    i_list=[x for x in range(max_val+1)]  # values that i and j can take
#   print(i_list)
    i_list = np.linspace(0, 20, num = 21,dtype=int)
    print("Range of i and j is : ",i_list,"Type returned", type(i_list))
    values_list=list(map(value_func,i_list))
    print(values_list)
    return values_list

values_list=value_gen(20)

Range of i and j is :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20] Type returned <class 'numpy.ndarray'>
[-2.0, -1.8, -1.6, -1.4, -1.2, -1.0, -0.7999999999999998, -0.5999999999999999, -0.3999999999999999, -0.19999999999999996, 0.0, 0.20000000000000018, 0.40000000000000036, 0.6000000000000001, 0.8000000000000003, 1.0, 1.2000000000000002, 1.4000000000000004, 1.6, 1.8000000000000003, 2.0]


In [52]:
# def random_train_samples(value_list):
#     train_pts=np.random.choice(value_list,size=441,replace=True)
#     print(train_pts)
#     print("Shape is:",train_pts.shape)
#     random_train_samples=np.array((np.random.choice(value_list,size=(1,2),replace=True))for i in range(442))
#     print(random_train_samples)
#     print("Shape is:",random_train_samples.shape)
#     return random_train_samples
# random_train_samples(values_list)
import random
def random_train_samples(value_list):
    empty=np.empty((21,21),dtype=tuple)
#     print(empty)
    for i in range(21):
        for j in range(21):
            x_1=random.choice(value_list)
            x_2=random.choice(value_list)
            empty[i,j]=(x_1,x_2)
    print(empty)
    print(empty.shape)
    return empty
random_train_samples(values_list)

[[(-1.0, 0.8000000000000003) (-0.3999999999999999, -0.3999999999999999)
  (-1.8, -0.3999999999999999) (1.0, 1.4000000000000004)
  (1.0, -0.19999999999999996) (-1.2, 0.8000000000000003)
  (-1.6, 0.20000000000000018) (-0.5999999999999999, -1.2) (-1.0, -1.6)
  (0.6000000000000001, 0.20000000000000018)
  (0.40000000000000036, 0.40000000000000036)
  (0.8000000000000003, 0.8000000000000003) (0.40000000000000036, -1.0)
  (-1.6, 1.8000000000000003) (0.20000000000000018, -1.8)
  (-1.4, -0.5999999999999999) (0.40000000000000036, -0.7999999999999998)
  (1.8000000000000003, -0.5999999999999999) (-1.4, -1.4)
  (-1.8, 0.20000000000000018) (0.0, 1.2000000000000002)]
 [(0.0, -1.6) (-1.2, 1.0) (-1.2, 1.0)
  (1.2000000000000002, -0.19999999999999996)
  (1.8000000000000003, 0.6000000000000001) (-0.19999999999999996, -1.4)
  (1.4000000000000004, -1.8) (2.0, -1.8) (1.8000000000000003, 1.0)
  (-0.3999999999999999, -2.0) (1.8000000000000003, 0.0) (-1.4, 2.0)
  (1.6, 0.20000000000000018) (-1.4, 0.600000000000

array([[(-1.0, 0.8000000000000003),
        (-0.3999999999999999, -0.3999999999999999),
        (-1.8, -0.3999999999999999), (1.0, 1.4000000000000004),
        (1.0, -0.19999999999999996), (-1.2, 0.8000000000000003),
        (-1.6, 0.20000000000000018), (-0.5999999999999999, -1.2),
        (-1.0, -1.6), (0.6000000000000001, 0.20000000000000018),
        (0.40000000000000036, 0.40000000000000036),
        (0.8000000000000003, 0.8000000000000003),
        (0.40000000000000036, -1.0), (-1.6, 1.8000000000000003),
        (0.20000000000000018, -1.8), (-1.4, -0.5999999999999999),
        (0.40000000000000036, -0.7999999999999998),
        (1.8000000000000003, -0.5999999999999999), (-1.4, -1.4),
        (-1.8, 0.20000000000000018), (0.0, 1.2000000000000002)],
       [(0.0, -1.6), (-1.2, 1.0), (-1.2, 1.0),
        (1.2000000000000002, -0.19999999999999996),
        (1.8000000000000003, 0.6000000000000001),
        (-0.19999999999999996, -1.4), (1.4000000000000004, -1.8),
        (2.0, -1.8), (